In [ ]:
!pip install -U transformers datasets accelerate peft bitsandbytes safetensors

In [ ]:
import transformers
print(transformers.__version__)

In [ ]:
from google.colab import files

uploaded = files.upload()
print(uploaded.keys())

In [ ]:
import os
from dataclasses import dataclass
from typing import Dict, List, Union

import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
)
from peft import LoraConfig, get_peft_model, TaskType


MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"
DATA_PATH = "data.jsonl"
OUTPUT_DIR = "./structmath-lora-model"

MAX_LENGTH = 512
BATCH_SIZE = 1
NUM_EPOCHS = 3
LEARNING_RATE = 2e-4
WARMUP_STEPS = 50
GRAD_ACCUM_STEPS = 8


tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
)


lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()



raw_ds = load_dataset("json", data_files=DATA_PATH, split="train")

def format_example(example):
    theorem_text = example["input"]
    json_target = example["output"]

    prompt = (
        "You are an assistant that extracts structured information from"
        " mathematical theorems.\n\n"
        "Task: Given the following theorem, output a JSON object with fields:\n"
        "- type\n- id\n- name (optional)\n- assumptions (list of strings)\n"
        "- conclusion (string)\n\n"
        "Theorem:\n"
        f"{theorem_text}\n\n"
        "JSON:\n"
        "<json>\n"
    )

    target = json_target + "\n</json>"

    return {"prompt": prompt, "target": target}

formatted_ds = raw_ds.map(format_example, remove_columns=raw_ds.column_names)

ds = formatted_ds.train_test_split(test_size=0.1, seed=42)
train_ds = ds["train"]
val_ds = ds["test"]





def tokenize_fn(example):
    prompt_ids = tokenizer(
        example["prompt"],
        add_special_tokens=False,
    )["input_ids"]

    target_ids = tokenizer(
        example["target"],
        add_special_tokens=False,
    )["input_ids"]

    input_ids = prompt_ids + target_ids

    labels = [-100] * len(prompt_ids) + target_ids

    input_ids = input_ids[:MAX_LENGTH]
    labels = labels[:MAX_LENGTH]

    attention_mask = [1] * len(input_ids)

    return {
        "input_ids": input_ids,
        "labels": labels,
        "attention_mask": attention_mask,
    }

tokenized_train = train_ds.map(
    tokenize_fn,
    batched=False,
    remove_columns=["prompt", "target"],
)

tokenized_val = val_ds.map(
    tokenize_fn,
    batched=False,
    remove_columns=["prompt", "target"],
)

tokenized_train.set_format(type="torch")
tokenized_val.set_format(type="torch")


data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)



training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM_STEPS,
    num_train_epochs=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    warmup_steps=WARMUP_STEPS,
    logging_steps=50,
    fp16=True,
    save_total_limit=3,
    report_to=[],
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
)


trainer.train()

trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
model.save_pretrained(OUTPUT_DIR)



In [ ]:
model.save_pretrained(OUTPUT_DIR)


In [ ]:
!zip -r structmath-lora-model.zip structmath-lora-model


In [ ]:
from google.colab import files
files.download("structmath-lora-model.zip")

In [ ]:
import os
import zipfile
from google.colab import files

LORA_ZIP_NAME = "structmath-lora-model.zip"
LORA_DIR = "structmath-lora-model"

uploaded = files.upload()

if LORA_ZIP_NAME not in uploaded:
    LORA_ZIP_NAME = list(uploaded.keys())[0]

if os.path.isdir(LORA_DIR):
    import shutil
    shutil.rmtree(LORA_DIR)

with zipfile.ZipFile(LORA_ZIP_NAME, 'r') as zf:
    zf.extractall(".")

if not os.path.isdir(LORA_DIR):
    raise RuntimeError(f" Expected directory '{LORA_DIR}' but it was not created.")

contents = os.listdir(LORA_DIR)

In [ ]:
import re
import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

BASE_MODEL = "mistralai/Mistral-7B-Instruct-v0.2"
LORA_DIR = "./structmath-lora-model"  # adjust if needed

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(LORA_DIR)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=False,
)

model = PeftModel.from_pretrained(
    base_model,
    LORA_DIR,
    low_cpu_mem_usage=False,
    is_trainable=False,
)

model.to(DEVICE)
model.eval()


def build_prompt(theorem_text: str) -> str:
    return (
        "You are an assistant that extracts structured information from"
        " mathematical theorems.\n\n"
        "Task: Given the following theorem, output a JSON object with fields:\n"
        "- type\n- id\n- name (optional)\n- assumptions (list of strings)\n"
        "- conclusion (string)\n\n"
        "Theorem:\n"
        f"{theorem_text}\n\n"
        "JSON:\n"
        "<json>\n"
    )


def extract_json_block(text: str):
    m = re.search(r"<json>\s*(\{.*?\})\s*</json>", text, re.S)
    if not m:
        return None
    return m.group(1).strip()


def test_llm(theorem_text: str):
    prompt = build_prompt(theorem_text)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.0,
            do_sample=False,
        )

    full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("\n=== Full model output ===")
    print(full_text)

    json_block = extract_json_block(full_text)
    if json_block is None:
        print("\n No <json>...</json> block found.")
        return

    try:
        parsed = json.loads(json_block)
    except json.JSONDecodeError as e:
        print("\n JSON parse error:", e)
        print("Raw JSON block:")
        print(json_block)
        return

    print("\n Parsed JSON:")
    print(json.dumps(parsed, indent=2))


In [ ]:
test_llm("For an elliptic differential operator on a compact manifold, its analytical index equals its topological index.")
test_llm("Let f be differentiable on the interval (a,b). Then f is continuous on (a,b).")
test_llm("Let T: V → W be a linear transformation. If T is injective, then T(v) = 0 implies v = 0.")
test_llm("In a group G, the identity element is unique.")
test_llm("Let (X, d) be a metric space. If X is compact, then every sequence in X has a convergent subsequence.")

